In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.action_chains import ScrollOrigin
from selenium.webdriver.common.action_chains import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from time import sleep
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import random

import datetime 
# import utils

import os 
import logging

import requests 
import json
import datetime 
import sys

def setting_log():
    '''
        해당 시스템 log 설정 
    '''
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y_%m_%d_%H_%M_%S")
    logging.basicConfig(filename=f"./save_log/{formatted_datetime}.log",
                    format='%(asctime)s %(levelname)s:%(message)s',
                    level=logging.INFO,
                    datefmt='%Y/%m/%d %I:%M:%S %p', )

# 하나의 동을 끝내면 저장하는 폴더
folder_path = "./record_info"
log_path = "./save_log"

# 크롤링 저장 폴더가 없으면 생성
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"{folder_path} 폴더가 생성되었습니다.")
else:
    print(f"{folder_path} 폴더가 이미 존재합니다.")

# 로그 폴더가 없으면 생성 
if not os.path.exists(log_path):
    os.makedirs(log_path)
    print(f"{log_path} 폴더가 생성되었습니다.")
else:
    print(f"{log_path} 폴더가 이미 존재합니다.")


# log 설정 세팅
setting_log() 

./record_info 폴더가 이미 존재합니다.
./save_log 폴더가 이미 존재합니다.


In [2]:
def init_():
    driver = webdriver.Chrome()
    driver.implicitly_wait(1) # 암시적 대기 시간 설정 
    driver.set_page_load_timeout(20) # 페이지 로드 대기 시간 설정 -> 해당 시간이 지나면 TimeException 발생 

    driver.get("https://www.disco.re/") # url 실행
    try:
        close = driver.find_element(By.CLASS_NAME, "disco-welcome-button.disco-welcome-close")
        ActionChains(driver).click(close).perform()
        return driver
    except:
        return driver

def settings(driver , text = "제주도 서귀포시 남원읍"):
    '''
        text : 검색하고자 하는 주소, 시 구 동까지 적어주면 좋음 
    
    '''
    # 1. 돋보기 누르기 
    driver.find_element(By.ID, "dsv_search_btn").click()  # ID로 입력란 찾기
    time.sleep(1)

    # 2. 검색창 입력 
    search_input = driver.find_element(By.ID, "top_search_ds_input")  # ID로 입력란 찾기
    search_input.click()
    search_input.send_keys(f"{text}")  # input_value를 작성란에 입력
    time.sleep(0.5)

    # 3. 첫번째 row 클릭 
    driver.find_element(By.CLASS_NAME, "autocomplete-suggestion").click()  # ID로 입력란 찾기
    # 4. 매물 버튼 클릭 
    sale_button = driver.find_element(By.XPATH, '//*[@data-value="sale"]')
    sale_button.click()

    # 거래방식 -> 매물로 조정
    driver.find_element(By.ID, "dealDropdown").click() 
    driver.find_element(By.XPATH, '//*[@class="dropdown-content show"]//li[@data-value="0"]').click()
    
def find_other_location(driver):
    # 무한 스크롤을 위해 스크롤의 범위 설정
    scrollable_element = driver.find_element(By.ID, "dsv_filter_tabmenu_sub")

    while True:
        for _ in range(100):
            driver.execute_script("arguments[0].scrollIntoView();", scrollable_element) # 스크롤 

        # 지역이 바뀌는 경우까지 스크롤을 하였는가 체크 
        # 0이 되는 시점이 다른 지역을 찾는 경우임
        element = driver.find_elements(By.XPATH, '//*[@data-area-order="0"]')
        # 다른 지역이 나오기 시작 
        if element != []:
            break
    
    return driver.find_elements(By.XPATH, '//*[@data-area-order="1"]')

# def start_crawling(driver , find_list):
#     results = []
    
#     for now_row in find_list:
#         temp = now_row.find_element(By.CLASS_NAME,"feed-content.feed-clickable").get_attribute("onclick")
#         driver.execute_script(temp)
    
#         time.sleep(0.5)
#             # 주소, 금액 정보 얻기 
#         address = driver.find_element(By.CLASS_NAME, "dsv-mc-address").text
#         cost = driver.find_element(By.CLASS_NAME, "dsv-mc-price").text

#         # 정보 글고 뒤로 이동 
#         driver.find_element(By.CLASS_NAME, "dsc-head-left-align").click()
#         results.append([address,cost])
#         time.sleep(0.5)
    
#     return results


def onclick_url_extract(find_list):
    js_url = []
    for i in find_list:
        text = i.find_element(By.CLASS_NAME,"feed-content.feed-clickable").get_attribute("onclick")
        print(text)
        js_url.append(text)
        
    return js_url

def start_crawling(driver, js_url):
    results = []
    for script in js_url:
        driver.execute_script(script)
        
        time.sleep(0.5)
            # 주소, 금액 정보 얻기 
        address = driver.find_element(By.CLASS_NAME, "dsv-mc-address").text
        cost = driver.find_element(By.CLASS_NAME, "dsv-mc-price").text

        # 정보 글고 뒤로 이동 
        # driver.find_element(By.CLASS_NAME, "dsc-head-left-align").click()
        results.append([address,cost])
        time.sleep(0.5)
    
    return results
    
# driver 정의
driver = init_()
# 크롤링 할 수 있도록 '주소' 세팅 
settings(driver, "서울특별시 강남구 논현동")
# settings(driver, "서울특별시 양천구 신월동")


# # 크롤링 해야 할 리스트 추출 
find_list = find_other_location(driver)

# onclick url find
# 실질적 정보 추출 시작 
# results = start_crawling(driver , find_list)

In [3]:
js_url = onclick_url_extract(find_list)

duser_request2(982, 'lufqz5vn','',''); show_detail_estate_data('sale', 'lufqz5vn',true);
duser_request2(982, 'flrhyof6','',''); show_detail_estate_data('sale', 'flrhyof6',true);
duser_request2(982, 'u1yzyul5','',''); show_detail_estate_data('sale', 'u1yzyul5',true);
duser_request2(982, 'gi2q2ufl','',''); show_detail_estate_data('sale', 'gi2q2ufl',true);
duser_request2(982, 'gud1byju','',''); show_detail_estate_data('sale', 'gud1byju',true);
duser_request2(982, 'ym8e76cz','',''); show_detail_estate_data('sale', 'ym8e76cz',true);
duser_request2(982, 'eiairzsx','',''); show_detail_estate_data('sale', 'eiairzsx',true);
duser_request2(982, 'azs5jimn','',''); show_detail_estate_data('sale', 'azs5jimn',true);
duser_request2(982, 'r8rt369f','',''); show_detail_estate_data('sale', 'r8rt369f',true);
duser_request2(982, 'xhrgwmtz','',''); show_detail_estate_data('sale', 'xhrgwmtz',true);
duser_request2(982, 'b7jwh4iu','',''); show_detail_estate_data('sale', 'b7jwh4iu',true);
duser_request2(982, '

In [4]:
def start_crawling(driver, js_url):
    results = []
    for index, script in enumerate(js_url):
        driver.execute_script(script)
        
        time.sleep(0.5)
        # 주소, 금액 정보 얻기 
        address = driver.find_element(By.CLASS_NAME, "dsv-mc-address").text
        cost = driver.find_element(By.CLASS_NAME, "dsv-mc-price").text

        # 정보 글고 뒤로 이동 
        # driver.find_element(By.CLASS_NAME, "dsc-head-left-align").click()
        print(index , address)
        results.append([address,cost])
    
    return results

results = start_crawling(driver , js_url)

0 서울특별시 강남구 논현동 136-40
1 서울특별시 강남구 논현동 111-31
2 서울특별시 강남구 논현동 111-28
3 서울특별시 강남구 논현동 111-23
4 서울특별시 강남구 논현동 102-19
5 서울특별시 강남구 논현동 101-20
6 서울특별시 강남구 논현동 206-17
7 서울특별시 강남구 논현동 158-34
8 서울특별시 강남구 논현동 188-23
9 서울특별시 강남구 논현동 37-9
10 서울특별시 강남구 논현동 147-1
11 서울특별시 강남구 논현동 11-13
12 서울특별시 강남구 논현동 171-23
13 서울특별시 강남구 논현동 13-26
14 서울특별시 강남구 논현동 57-9
15 서울특별시 강남구 논현동 37-9
16 서울특별시 강남구 논현동 16-3
17 서울특별시 강남구 논현동 205-5
18 서울특별시 강남구 논현동 37-9
19 서울특별시 강남구 논현동 184-3
20 서울특별시 강남구 논현동 161-13
21 서울특별시 강남구 논현동 182-27
22 서울특별시 강남구 논현동 181-10
23 서울특별시 강남구 논현동 104-32
24 서울특별시 강남구 논현동 270
25 서울특별시 강남구 논현동 242-35
26 서울특별시 강남구 논현동 242-25
27 서울특별시 강남구 논현동 242-1
28 서울특별시 강남구 논현동 242
29 서울특별시 강남구 논현동 240-3
30 서울특별시 강남구 논현동 37-9
31 서울특별시 강남구 논현동 110-24
32 서울특별시 강남구 논현동 103-7
33 서울특별시 강남구 논현동 103-6
34 서울특별시 강남구 논현동 101-1
35 서울특별시 강남구 논현동 190-5
36 서울특별시 강남구 논현동 98-10
37 서울특별시 강남구 논현동 65-18
38 서울특별시 강남구 논현동 23-5
39 서울특별시 강남구 논현동 20-9
40 서울특별시 강남구 논현동 206-17
41 서울특별시 강남구 논현동 205-5
42 서울특별시 강남구 논현동 269-3
43 서울특별시 강남구 논현

TimeoutException: Message: timeout: Timed out receiving message from renderer: 20.000
  (Session info: chrome=136.0.7103.93)
Stacktrace:
	GetHandleVerifier [0x00007FF636BECF65+75717]
	GetHandleVerifier [0x00007FF636BECFC0+75808]
	(No symbol) [0x00007FF6369B8F9A]
	(No symbol) [0x00007FF6369A64EC]
	(No symbol) [0x00007FF6369A61DA]
	(No symbol) [0x00007FF6369A3E8A]
	(No symbol) [0x00007FF6369A483F]
	(No symbol) [0x00007FF6369B33AE]
	(No symbol) [0x00007FF6369C97E1]
	(No symbol) [0x00007FF6369D091A]
	(No symbol) [0x00007FF6369A4FAD]
	(No symbol) [0x00007FF6369C94D0]
	(No symbol) [0x00007FF636A5F276]
	(No symbol) [0x00007FF636A37153]
	(No symbol) [0x00007FF636A00421]
	(No symbol) [0x00007FF636A011B3]
	GetHandleVerifier [0x00007FF636EED74D+3223469]
	GetHandleVerifier [0x00007FF636EE7CF2+3200338]
	GetHandleVerifier [0x00007FF636F05B23+3322755]
	GetHandleVerifier [0x00007FF636C06A3A+180890]
	GetHandleVerifier [0x00007FF636C0E13F+211359]
	GetHandleVerifier [0x00007FF636BF52B4+109332]
	GetHandle

In [ ]:
driver.find_element(By.CLASS_NAME, "dsv-mc-price").text

'매매 190억(2335.33만원/m2)'

In [ ]:
driver.execute_script(js_url[630])

In [7]:
js_url[618]

"duser_request2(982, 'lut6jqbs','',''); show_detail_estate_data('sale', 'lut6jqbs',true);"

In [37]:
# temp = now_row.find_element(By.CLASS_NAME, "feed-content.feed-clickable").get_attribute("onclick")

# 2. JavaScript 코드로 두 함수 호출 실행
script = """
duser_request2(982, '48p1bctr','',''); 
show_detail_estate_data('sale', '48p1bctr', true);
"""
driver.execute_script(script)

In [20]:
# 실질적 정보 추출 시작 
results = start_crawling(driver , find_list)

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=136.0.7103.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7B146CF65+75717]
	GetHandleVerifier [0x00007FF7B146CFC0+75808]
	(No symbol) [0x00007FF7B1238F9A]
	(No symbol) [0x00007FF7B124F3D4]
	(No symbol) [0x00007FF7B124DEA3]
	(No symbol) [0x00007FF7B12416C9]
	(No symbol) [0x00007FF7B123F54F]
	(No symbol) [0x00007FF7B1243A0C]
	(No symbol) [0x00007FF7B1243ADF]
	(No symbol) [0x00007FF7B128ED77]
	(No symbol) [0x00007FF7B128F77C]
	(No symbol) [0x00007FF7B1281D7C]
	(No symbol) [0x00007FF7B12B73BF]
	(No symbol) [0x00007FF7B1281C46]
	(No symbol) [0x00007FF7B12B7590]
	(No symbol) [0x00007FF7B12DF39C]
	(No symbol) [0x00007FF7B12B7153]
	(No symbol) [0x00007FF7B1280421]
	(No symbol) [0x00007FF7B12811B3]
	GetHandleVerifier [0x00007FF7B176D